<a href="https://colab.research.google.com/github/joselopez11c/mi-primer-proyecto/blob/main/remates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pdfplumber
!pip install curlify

import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive
import os
import pdfplumber
import tempfile
import curlify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for curlify: filename=curlify-2.2.1-py3-none-any.whl size=2666 sha256=8f8b5782eb57e510765757979a968237216f72ea1c0eab50f2bc887f447b9c49
  Stored in directory: /root/.cache/pip/wheels/01/e8/2f/e0e986ab1e8956d5ad10aabc059767c822d6114ed295a28dd5
Successfully built curlify


In [60]:
url = 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml'
urlLogin = 'https://remaju.pj.gob.pe/remaju/pages/seguridad/login.xhtml'

headers = {
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,es;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Faces-Request': 'partial/ajax',
    'Origin': 'https://remaju.pj.gob.pe',
    'Referer': 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

# Realizar la solicitud GET
response = requests.get(urlLogin, headers=headers)
cookie_header = 'JSESSIONID=' + response.cookies.get_dict().get('JSESSIONID')
headers['Cookie'] = cookie_header


def fetch_page(first_row):
  dataPagination = {
      'javax.faces.partial.ajax': 'true',
      'javax.faces.source': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.execute': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.render': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.behavior.event': 'page',
      'javax.faces.partial.event': 'page',
      'formBuscarRemateExterno:listaRemate_pagination': 'true',
      'formBuscarRemateExterno:listaRemate_skipChildren': 'true',
      'formBuscarRemateExterno:listaRemate_first': str(first_row),
      'formBuscarRemateExterno:listaRemate_rows': '12',
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
  }
  response = requests.post(url, headers=headers, data=dataPagination)
  return response

def downloadFile(idFile):

  url = "https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml"

  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': cookie_header
  }

  dataFile = {
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
      'formBuscarRemateExterno:listaRemate:0:j_idt171': '',
      'javax.faces.ViewState': view_state_value
  }

  response = requests.post(url, headers=headers, data=dataFile)
  #if response.status_code == 200:
  #  with open(file_path, 'wb') as file:
  #      file.write(response.content)
  #      print(f"Archivo guardado en {file_path}")
  #else:
  #    print(f"Error descargando el archivo en {file_path}")
  return response


page = fetch_page(0) ####################################################### LLAMADA A LA PAGINA

soup = BeautifulSoup(page.content, 'xml')

update_tag = soup.find('update', {'id': 'j_id1:javax.faces.ViewState:0'})
view_state_value = update_tag.text

cdata_content = soup.find('update').text

# Ahora parseamos el contenido del CDATA, que es HTML
soup_html = BeautifulSoup(cdata_content, 'html.parser')

# Buscar todos los div con la clase 'card azul'
cards = soup_html.find_all('div', class_='card azul')

data = []
for card in cards:
    # Obtener número de remate y convocatoria
    remate_info = card.find('span', class_='text-bold label-danger h6').get_text(strip=True) if card.find('span', class_='text-bold label-danger h6') else 'No disponible'

    # Dividir la información de remate en número y convocatoria
    if remate_info != 'No disponible':
        parts = remate_info.split(" - ")
        remate_number = parts[0] if len(parts) > 0 else 'No disponible'
        convocatoria = parts[1] if len(parts) > 1 else 'No disponible'
    else:
        remate_number = 'No disponible'
        convocatoria = 'No disponible'

    # Ubicación, buscando la etiqueta específica y extrayendo el texto siguiente
    location_tag = card.find('i', class_='fa fa-map-marker')
    location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'

    # Descripción del inmueble, buscando el contenedor específico
    description = card.find('div', class_='texto-info-scroll').get_text(strip=True) if card.find('div', class_='texto-info-scroll') else 'No disponible'

    # Buscar todos los scripts dentro del card
    scripts = card.find_all('script')
    script_id = scripts[-1].get('id')[:-2] if scripts and scripts[-1].get('id').endswith('_s') else 'Script ID no encontrado'

    # Guardar los datos extraídos en la lista
    data.append({
        'Numer': remate_number,
        'Convocation': convocatoria,
        'Location': location,
        'Description': description,
        'Script ID': script_id,
        'View Code' : view_state_value,
        'Cookie header' : cookie_header
    })

# Crear un DataFrame con los datos recolectados
df = pd.DataFrame(data)

print(cookie_header)

df.head()

JSESSIONID="9ShjbgZlottbSspJfwMJbf-E6Cs8fWGChXgQnabH.P-Nodo5:SRemaju7-3"


<ipython-input-60-1b9ef5cdfc0e>:100: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'


,Numer,Convocation,Location,Description,Script ID,View Code,Cookie header
0,Remate N° 14066,PRIMERA CONVOCATORIA,SAN VICENTE DE CAÑETE,INMUEBLE UBICADO URBANIZACION BETO JOSE MARÍA ...,formBuscarRemateExterno:listaRemate:0:j_idt171,5808906128221864113:8126517686487157815,"JSESSIONID=""9ShjbgZlottbSspJfwMJbf-E6Cs8fWGChX..."
1,Remate N° 14065,PRIMERA CONVOCATORIA,SAN LUIS DE SHUARO,INMUEBLE UBICADO EN EL SECTOR 1 MZ. E LOTE 12 ...,formBuscarRemateExterno:listaRemate:1:j_idt171,5808906128221864113:8126517686487157815,"JSESSIONID=""9ShjbgZlottbSspJfwMJbf-E6Cs8fWGChX..."
2,Remate N° 14064,PRIMERA CONVOCATORIA,COMAS,PREDIO RUSTICO CON 0.8602 HECTAREAS,formBuscarRemateExterno:listaRemate:2:j_idt171,5808906128221864113:8126517686487157815,"JSESSIONID=""9ShjbgZlottbSspJfwMJbf-E6Cs8fWGChX..."
3,Remate N° 14063,PRIMERA CONVOCATORIA,TAMBOPATA,PREDIO URBANO UBICADO EN JR. SISA MZ. D LOTE 0...,formBuscarRemateExterno:listaRemate:3:j_idt171,5808906128221864113:8126517686487157815,"JSESSIONID=""9ShjbgZlottbSspJfwMJbf-E6Cs8fWGChX..."
4,Remate N° 14062,PRIMERA CONVOCATORIA,PIURA,"DESCRIPCION: DEPARTAMENTO N° 201, EDIFICIO “L”...",formBuscarRemateExterno:listaRemate:4:j_idt171,5808906128221864113:8126517686487157815,"JSESSIONID=""9ShjbgZlottbSspJfwMJbf-E6Cs8fWGChX..."


In [61]:
for index, row in df.iterrows():
  file_name = row['Numer'] + '.pdf'
  #file_name = 'aaa' + '.pdf'
  #response = downloadFile(row['Script ID'], view_state_value, 'aaa.pdf')
  response = downloadFile(row['Script ID'])

  # Check if the response contains content
  if response.ok and 'application/pdf' in response.headers.get('Content-Type', ''):
      # Saving the PDF to a local file
      pdf_path = 'downloaded_file.pdf'
      with open(pdf_path, 'wb') as f:
          f.write(response.content)

      # Reading the PDF using pdfplumber
      with pdfplumber.open(pdf_path) as pdf:
          text = ""
          for page in pdf.pages:
              text += page.extract_text() or ''  # Append extracted text, use '' if None is returned

      # Print the extracted text
      print(text)

      # Optionally, remove the PDF file if not needed
      os.remove(pdf_path)

  else:
      print("Failed to download a valid PDF file.")


REMATE JUDICIAL ELECTRÓNICO N° 14066
CONVOCATORIA DE REMATE
CONVOCATORIA A REMATE PÚBLICO.- En los seguidos por BANCO DE CREDITO DEL PERU, contra CORPORACION
EDUCATIVA MEZA HERMANOS SOCIEDAD ANONIMA CERRADA, PATRICIA CAROLINA MEZA MANYARI, sobre proceso de
EJECUCION DE GARANTIAS, en el Expediente Judicial N° 00117-2023-0-0801-JR-CI-01, tramitado ante el 1° JUZGADO CIVIL -
Sede Central DE CAÑETE, a cargo del Juez DR(A). JULIO CESAR PARVINA MELGAR y del Especialista Legal DR(A).
MELGAREJO FERNANDEZ LUIS ALBERTO, conforme a lo dispuesto en la resolución N° 5, que ordena se proceda al remate
judicial electrónico a través del aplicativo informático REM@JU, conforme a las disposiciones de la Ley N° 30229.
DATOS DEL BIEN OBJETO DE REMATE
EL CIEN POR CIENTO (100 %) DE LAS ACCIONES Y DERECHOS SOBRE EL BIEN INMUEBLE UBICADO EN
URBANIZACION BETO JOSE MARÍA ESCRIVA DE BALAGUER, II ETAPA, PARQUE RESIDENCIAL LOS REYES MZ. D, LOTE
20, DISTRITO DE SAN VICENTE DE CAÑETE, PROVINCIA DE CAÑETE Y DEPARTAME

In [98]:
drive.mount('/content/drive')
# Ruta de la carpeta dentro de Google Drive
folder_path = '/content/drive/My Drive/Remates'
os.makedirs(folder_path, exist_ok=True)  # Crea la carpeta si no existe
print(cookie_header)

for index, row in df.iterrows():
  file_name = f"file_{index}.pdf"  # Asume que los archivos son PDFs
  file_path = os.path.join(folder_path, file_name)
  #print(row['Script ID'] + " " + view_state_value)
  downloadFile(row['Script ID'], view_state_value, file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
JSESSIONID="K6YF_5oTcqKrbPV984w4QzhNRNEUfByLzpIkecOP.P-Nodo5:SRemaju7-3"
cookie_header: JSESSIONID="K6YF_5oTcqKrbPV984w4QzhNRNEUfByLzpIkecOP.P-Nodo5:SRemaju7-3"


TypeError: can only concatenate str (not "dict") to str

In [22]:
#PARA LEER EL ARCHIVO PDF Y PASARLO A TEXT

from google.colab import files

# Cargar archivo PDF desde tu sistema local
uploaded = files.upload()

# Obtener el nombre del archivo cargado
file_name = list(uploaded.keys())[0]

# Leer el texto del PDF con pdfplumber
with pdfplumber.open(file_name) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()

# Imprimir el texto extraído
print(text)

KeyboardInterrupt: 

In [52]:
import requests

url = "https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml"

#view_state_value = '427971179364553855:5887553199467774138'
#cookie_header = 'JSESSIONID="5wGp20STeIrqNdfQA40Zmc4gaoElP7nglImJ3-3X.P-Nodo5:SRemaju7-3";'

headers = {
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': cookie_header
}

data = {
    'formBuscarRemateExterno': 'formBuscarRemateExterno',
    'formBuscarRemateExterno:listaRemate:0:j_idt171': '',
    'javax.faces.ViewState': view_state_value
}

print(headers)
print(data)

response = requests.post(url, headers=headers, data=data)

print(response)


# Check if the response contains content
if response.ok and 'application/pdf' in response.headers.get('Content-Type', ''):
    # Saving the PDF to a local file
    pdf_path = 'downloaded_file.pdf'
    with open(pdf_path, 'wb') as f:
        f.write(response.content)

    # Reading the PDF using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ''  # Append extracted text, use '' if None is returned

    # Print the extracted text
    print(text)

    # Optionally, remove the PDF file if not needed
    os.remove(pdf_path)

else:
    print("Failed to download a valid PDF file.")




{'Accept': 'application/xml, text/xml, */*; q=0.01', 'Content-Type': 'application/x-www-form-urlencoded', 'Cookie': 'JSESSIONID="3RjVEwaIUV-sHUYWp_1cgaHUEPJF76ThZYaV4ZNu.P-Nodo5:SRemaju7-3"'}
{'formBuscarRemateExterno': 'formBuscarRemateExterno', 'formBuscarRemateExterno:listaRemate:0:j_idt171': '', 'javax.faces.ViewState': '-4607616407865862175:1151084557732008850'}
<Response [200]>
REMATE JUDICIAL ELECTRÓNICO N° 14066
CONVOCATORIA DE REMATE
CONVOCATORIA A REMATE PÚBLICO.- En los seguidos por BANCO DE CREDITO DEL PERU, contra CORPORACION
EDUCATIVA MEZA HERMANOS SOCIEDAD ANONIMA CERRADA, PATRICIA CAROLINA MEZA MANYARI, sobre proceso de
EJECUCION DE GARANTIAS, en el Expediente Judicial N° 00117-2023-0-0801-JR-CI-01, tramitado ante el 1° JUZGADO CIVIL -
Sede Central DE CAÑETE, a cargo del Juez DR(A). JULIO CESAR PARVINA MELGAR y del Especialista Legal DR(A).
MELGAREJO FERNANDEZ LUIS ALBERTO, conforme a lo dispuesto en la resolución N° 5, que ordena se proceda al remate
judicial electróni

b'%PDF-1.7\n%\xe2\xe3\xcf\xd3\n5 0 obj\n<</Filter/FlateDecode/Length 2038>>stream\nx\x9c\xa5XMo\xdbH\x12\xbd\xebW\x14\x06;\x80\x03\xc8\x1dv\xf3{O\xdb";\x1e\x1a2\xa9\x90\x94\x81\x04\xb9\xd0\x12\xe3e \x89\n%%\xeb\xf9U\xf9\x0b{\xdc\xc3\x1e\x16\xb9\xcdm\x8es\xce\x1f\xd8\xaan\xcaq\xecl(\xcc\xc6\x87\xc84\xd9|\xf5\xaa\xde\xab*\xbd\x1fq\xee2\xc7\x06\x0b\x7fl\x87\xd9\x1c\xec\x00|\xd7a^\x08\x8b\xf5\xe8y\xb2\xe6\x10\xb7\xa3\x97\xa3\xf7\x0f\xeft,\xe6\xfb\xe08\x9c\xb9>\xf8N\xc0\x84\xdd\xdf-\x8ewO\xca\xd1\xf3\x17\x1c\xb8\x80\xf2\xed\x88\xfb\x82\t\x0e\xbe\x8d\x0fxP.Gx\xb5\x1bY\xcc\x81\x8f\xa3\xb3\\]\xc9R\xc1\xe5<N\xa2DNAMUT\xe6\x9f\xd3$\xca \xfd\x04\xdc\xb1<\xefY\xf9n\xa4\xca\xa7\x07\x0b\xceY\xe8\x81\xcfm\xe6\x06\x8f\x0f\x8e\xb2\xf4:\x8bd\x99\xe5\x89\x84X\x81y\xd1\xd3\xa3B:\t\xc3\xf6\xfc\x90\xc2\xc2S,&,a\xe1\xf5\x05~\xf4,O \xe8\xc7\x07\xca\xfe<\x98\xfd6\x99"Vv\x0ej\x03\xabv\x07\xbb\xfa\xf6\xd0,\xf1\xc3\xb6\xed`"S\x8c\x03\xdf\x1e\xe5*NJ\xfa8\x85\x99\xca\xe7cX\xb4\x9b}WA\x94\xe5\xb3,\x97Q\x92\xa5\x84\x